# Análise Filogenética

### NW

In [12]:
def validar_prot(seq:str)->bool:
    assert type(seq)==str,'A sequência não foi fornecida em formato de string'
    lista_aminoacidos = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y','_']
    for aa in seq.upper():
        if aa not in lista_aminoacidos:
            return False
    
    return True

def formatar(x):
    if type(x) is int:
        return f"{x:>3d}"
    else:
        return f"{x:>3}"
    
def print_matrix(S1,S2,M):
    
    print(*list(" " + S1), sep="   ")
    for x2, L in zip(S2,M): #x2 caracter que vem da string S2
        LL=[formatar(x) for x in L]
        print(x2,*LL)
    print()

def score_sub_acid_nuc(x1:str,x2:str,match,mismatch)->int:
    if x1==x2:return match
    else: return mismatch

def score_subst_prot(nome_matriz:str, aa1:str, aa2:str) -> int:
    """
    Esta função abre e lê um destes ficheiros de texto: "BLOSSOM62.txt", "BLOSSOM80.txt", "PAM30.txt", "PAM70.txt" 
    retirando a matriz de substituição do ficheiro como uma matriz bidimensional.
    Depois navega na matriz de substituição obtida e retorna o score correspondente ao ????alinhamento???? de dois dos seguintes caracteres: 
    'A','R','N','D','C','Q','E','G','H','I','L','K','M','F','P','S','T','W','Y','V','B','J','Z','X','_',
    sendo que as letras representam os aminoácidos e o caracter '_' representa o codão stop.
    
    Parameters
    ----------
    nome_matriz:str
        Nome da matriz de substituição que se pretende que seja lida e utilizada para fornecer os scores de substituição

    aa1:str
        Aminoácido que vai ser procurado ao longo da primeira coluna da matriz de substituição

    aa2:str
        Aminoácido que vai ser procurado ao longo da primeira linha da matriz de substituição

    Returns
    -------
    score:int
        Score obtido para o alinhamento dos dois aminoácidos na matriz de substituição

    Raises
    ------
    AssertionError
        Quando o nome da matriz fornecido não corresponde a uma das matrizes aceites pela função:
        "BLOSSOM62", "BLOSSOM80", "PAM30", "PAM70"

    AssertionError
        Quando o caracter inserido correspondente ao 'aa1' não é válido como aminoácido ou codão stop

    AssertionError
        Quando o caracter inserido correspondente ao 'aa2' não é válido como aminoácido ou codão stop
    """
    assert nome_matriz.upper() in ["BLOSSOM62", "BLOSSOM80", "PAM30", "PAM70"],"A matriz de substituição escolhida é inválida, apenas aceita as matrizes 'BLOSSOM62', 'BLOSSOM80', 'PAM30', 'PAM70'"
    with open(nome_matriz.upper()+".txt", "r") as file:
        matriz = []
        for line in file:
            line = line.strip()
            linha = []
            valor = ""
            for elem in line:
                if elem == " ":
                    if len(valor) != 0:
                        linha.append(valor)
                        valor = ""
                else:
                    valor += elem
            linha.append(valor)
            matriz.append(linha)
    aa1, aa2 = aa1.upper(), aa2.upper()
    lista_aminoacidos = ['A','R','N','D','C','Q','E','G','H','I','L','K','M','F','P','S','T','W','Y','V','B','J','Z','X','_']
    assert aa1 in lista_aminoacidos,f"O caracter {aa1} não é válido como aminoácido ou codão stop"
    assert aa2 in lista_aminoacidos,f"O caracter {aa2} não é válido como aminoácido ou codão stop"
    coluna, linha = 0, 0
    for i in range(1,len(matriz)):
        if aa1 == matriz[i][0]:
            linha = i
            break
    for pos, elem in enumerate(matriz[0]):
        if elem == aa2:
            coluna = pos
            break
    score = matriz[linha][coluna]
    return int(score)

def reconstrucao(seq1,seq2,trace):
    aligned_seq1 = ''
    aligned_seq2 = ''
    linha=len(seq1)-1
    coluna=len(seq2)-1

    while trace[linha][coluna]!=0:
        if trace[linha][coluna] == '↖':
            aligned_seq1 = seq1[linha] + aligned_seq1
            aligned_seq2 = seq2[coluna] + aligned_seq2
            linha -= 1
            coluna -= 1
        elif trace[linha][coluna] == '↑':
            aligned_seq1 = seq1[linha] + aligned_seq1
            aligned_seq2 = '-' + aligned_seq2
            linha -= 1
        elif trace[linha][coluna] == '←':
            aligned_seq1 = '-' + aligned_seq1
            aligned_seq2 = seq2[coluna] + aligned_seq2
            coluna-= 1
    # print(aligned_seq1,aligned_seq2,sep='\n')
    return (aligned_seq1,aligned_seq2)

def needleman_wunsch(seq1:str,seq2:str,seq_tipo:str,gap:int=-4,matriz:str='Blossom62',match:int=2,mismatch:int=-1,matrizes=True ):
    """
    Realiza o alinhamento global de duas sequências recorrendo o algoritmo Needleman-Wunsch.

    Parameters
    ----------
    seq1 : str
      Primeira sequência
    seq2 : str
      Segunda sequência

    seq_tipo: str
        Identifica o tipo das sequências que se pretende alinhar

    gap: int
        Valor a utilizar para penalizar os espaçamentos. Por padrão considera-se o valor de penalização como -4

    matriz: str
        Matriz de substituição de aminoacidos a utilizar para realizar o alinhamento de proteínas

    match: int
        Valor a utilizar para quando as duas bases nucleicas são iguais. Por default é 2. Apenas para o alinhamento de DNA/RNA

    mismatch: int
        Valor a utilizar para quando as duas bases nucleicas são diferentes. Por default é -1. Apenas para o alinhamento de DNA/RNA

    matrizes: bool
        Indica 

    Returns
    ----------
    Tuple
     devolve um tuplo com a matriz de pontuação, a matriz de rastreamento e o alinhamento ótimo
    
    """
    assert seq_tipo.upper() in ['DNA','RNA','PROTEIN'], 'O tipo de sequência fornecido não é valido, apenas pode assumir o valor de DNA, RNA ou protein'
    
    if seq_tipo.upper()=='DNA' or seq_tipo.upper()=='RNA':
        assert seq_tipo in ['DNA','RNA'], 'A sequencia 1 não é um acido nucleico'
        assert seq_tipo in ['DNA','RNA'], 'A sequencia 2 não é um acido nucleico'
        assert seq_tipo==seq_tipo, 'As cadeias fornecidas não são do mesmo tipo'

    else:
        assert validar_prot(seq1), 'A sequencia 1 não é uma proteina'
        assert validar_prot(seq2), 'A sequencia 2 não é uma proteina'

    seq1='-'+seq1
    seq2='-'+seq2
    nrows = len(seq1)
    ncols = len(seq2)

    score_matrix=[[0]*ncols for linhas in range(nrows)] # iniciar a matriz score apenas com 0
    score_matrix[0]=[c*gap for c in range(ncols)] #prenchimento da primeira linha da matriz score
    for linha in range(1, nrows):score_matrix[linha][0] = linha * gap # preenchimento da primeira coluna da matriz score

    traceback_matrix = [[0] * ncols for linhas in range(nrows)] # iniciar a matriz traceback com 0
    traceback_matrix[0] =[0 if col==0 else '←' for col in range(ncols)] # prenchimento da primeira linha da matriz traceback
    for linha in range(1,nrows): traceback_matrix[linha][0]='↑' # preenchimento da primeira coluna da matriz traceback

    if seq_tipo.upper()=='DNA' or seq_tipo.upper()=='RNA': #alinhamento de acidos nucleicos
      for linha in range(1, nrows):
        for coluna in range(1, ncols):
          diagonal = score_matrix[linha - 1][coluna - 1] + score_sub_acid_nuc(seq1[linha],seq2[coluna],match,mismatch)
          cima = score_matrix[linha - 1][coluna] + gap
          esquerda = score_matrix[linha][coluna - 1] + gap

          max_score = max(diagonal, esquerda, cima)
          score_matrix[linha][coluna] = max_score

          if max_score == diagonal: traceback_matrix[linha][coluna] = '↖'
          elif max_score == cima: traceback_matrix[linha][coluna] = '↑'
          elif max_score == esquerda: traceback_matrix[linha][coluna] = '←'

    else: # alinhamento de proteinas
      for linha in range(1, nrows):
        for coluna in range(1, ncols):
          diagonal = score_matrix[linha - 1][coluna - 1] + score_subst_prot(matriz,seq1[linha],seq2[coluna])
          cima = score_matrix[linha - 1][coluna] + gap
          esquerda = score_matrix[linha][coluna - 1] + gap

          max_score = max(diagonal, esquerda, cima)
          score_matrix[linha][coluna] = max_score

          if max_score == diagonal: traceback_matrix[linha][coluna] = '↖'
          elif max_score == cima: traceback_matrix[linha][coluna] = '↑'
          elif max_score == esquerda: traceback_matrix[linha][coluna] = '←'

    if matrizes:
        print("Matriz de Pontuação:")
        print_matrix(seq2,seq1,score_matrix)
        print("Matriz de Rastreamento:")
        print_matrix(seq2,seq1,traceback_matrix)
        return score_matrix[-1][-1],reconstrucao(seq1,seq2,traceback_matrix)
    
    else:
        return score_matrix[-1][-1],reconstrucao(seq1,seq2,traceback_matrix)

In [13]:
# exemplo de alinhamento de DNA
needleman_wunsch("ACTGG", "AGA", 'DNA', matrizes=False)

(-5, ('ACTGG', 'A--GA'))

### UPGMA

1 Criar um cluster para cada sequência: L = {{si} : si ∈ S}

2 Matriz de distâncias entre os clusters é inicializada com a matriz de
distâncias entre as sequências

3 Enquanto existir mais do que um cluster

    1 Descobrir os clusters Ci e Ci com distância mínima entre eles
    
    2 Criar o cluster Ck = Ci ∪ Cj
    
    3 Adicionar o novo cluster a L e remover os anteriores:
    L = {Ck } ∪ L \ {Ci, Cj}
    4 Remover linhas e colunas i e j da tabela de distâncias
    
    5 Introduzir linha e coluna para k com as distâncias para o novo cluster na
    árvore, adicionar vértice ligando os nós referentes a Ci e Cj com valor de
    altura igual a d_{ij} / 2


- usar o NW para calcular os scores entre cada sequência
- usar o inverso dos scores do NW para obter a distância entre as matrizes e construir a matriz de distâncias

In [32]:
def criar_clusters(seqs:list[str]) -> list[{str}]:
    clusters = [{seq} for seq in seqs]
    return clusters

In [33]:
criar_clusters('ACT ACG ACA'.split())

[{'ACT'}, {'ACG'}, {'ACA'}]

In [23]:
def calc_dist(seq1 : str, seq2 : str) -> int:
    return -(needleman_wunsch(seq1, seq2, "DNA", matrizes=False)[0])

In [29]:
calc_dist('ACT','ACG')

-3

In [ ]:
def mat_dist_seqs(seqs : list[str]) -> dict[list[{str}]:int]:
    matriz = {}
    pass